# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

First install necessary libraries and import them.

In [ ]:
# Install Vertex AI LLM SDK
import sys
!{sys.executable} -m pip install google-cloud-aiplatform langchain pandas datasets google-api-python-client chromadb pypdf faiss-cpu transformers config --upgrade --user

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatVertexAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import VertexAIEmbeddings
from IPython.display import display, Markdown

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
import sys
!{sys.executable} -m pip install "langchain[docarray]"

In [ ]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [ ]:
embedding = VertexAIEmbeddings()
llm = ChatVertexAI(temperature=0.0)

index = VectorstoreIndexCreator(
    embedding=embedding,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [ ]:
response = index.query(llm=llm, question=query)

In [ ]:
display(Markdown(response))

In [ ]:
loader = CSVLoader(file_path=file)

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

In [ ]:
embed = embedding.embed_query("Hi my name is Harrison")

In [ ]:
print(len(embed))

In [ ]:
print(embed[:5])

In [ ]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embedding
)

In [ ]:
query = "Please suggest a shirt with sunblocking"

In [ ]:
docs = db.similarity_search(query)

In [ ]:
len(docs)

In [ ]:
docs[0]

In [ ]:
retriever = db.as_retriever()

In [ ]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [ ]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [ ]:
display(Markdown(response))

In [ ]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [ ]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [ ]:
response = qa_stuff.run(query)

In [ ]:
display(Markdown(response))

In [ ]:
response = index.query(query, llm=llm)

In [ ]:
display(Markdown(response))